# The Amazing Popularity of Spider-Man
by Leo Nguyen
<br/>
<br/>


![Imgur](https://i.imgur.com/VQCs4ff.jpg)
Comic book covers featuring Spider-Man throughout the ages.
<br/>
<br/>
<br/>
<br/>
Very quickly after his introduction in *Amazing Fantasy* \#15 (1962), **Spider-Man** has become Marvel Comics' most famous and beloved superhero with an enduring presence in popular culture across all types of media. I dived into several sources of data to investigate different contexts to his popularity.
<br/>
<br/>
<br/>
<br/>

In [1]:
# Note: If you are checking my code, just know that I have very little 
# experience with python & pandas so I'm sure it is very inelegant.

# libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import networkx as nx
from networkx.algorithms.community import asyn_fluidc
import csv
import warnings
warnings.filterwarnings("ignore")
import os

In [4]:
# Q1: Network analysis 
marvel_pairs = pd.read_csv(r'data\Marvel-AllPairs.tsv', sep="\t") 

marvel_pairs['Co-Featured'] = [w.replace(',', '') for w in marvel_pairs['Co-Featured']] # remove 2 values with commas
marvel_pairs['Co-Featured'] = marvel_pairs['Co-Featured'].astype(int)
marvel_pairs2 = marvel_pairs[marvel_pairs['Co-Featured'] >= 20]

costars =  marvel_pairs2.loc[marvel_pairs2[marvel_pairs2.Source == 'SPIDERMAN'].index, 'Target'] 
costars2 =  marvel_pairs2.loc[marvel_pairs2[marvel_pairs2.Target == 'SPIDERMAN'].index, 'Source'] 
costars3 = costars.values.tolist() + costars2.values.tolist() 
costars3.append('SPIDERMAN')

spider_pairs = marvel_pairs2.loc[(marvel_pairs2['Source'].isin(costars3)) & marvel_pairs2['Target'].isin(costars3)]
#spider_pairs.shape 
# OUTPUT: (1111, 3)

#make graph object for network
def marvel_pairs_to_graph(input_nodes):
    new_graph = nx.Graph()
    for row in input_nodes.iterrows(): 
        new_graph.add_node(row[1][0]) # Add first character
        new_graph.add_node(row[1][1]) # Add second character
        new_graph.add_edge(row[1][0],row[1][1],cofeatured=row[1][2])
    return new_graph
spider_graph = marvel_pairs_to_graph(spider_pairs)

# detect communities with Asynchronous Fluid Communities algorithm
n = 5
spider_asyn = asyn_fluidc(spider_graph, n)
af_communities = []
for i in range(0,n):
    af_communities.append(next(spider_asyn))

# write files
with open('data\spider_nodes.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(list(spider_graph.nodes()))
csvFile.close()
with open("data\spider_communities.txt", 'w') as f:
    for s in af_communities:
        f.write(str(s) + '\n')
f.close()
nx.write_gexf(spider_graph, "data\spider_graph2.gexf")
# this network went on to be visualized in Gephi

# check for most cofeatures

spider_pairs2 = spider_pairs.sort_values(by = ['Co-Featured'], ascending = False)
spider_pairs3 = spider_pairs2[(spider_pairs2['Source'] == 'SPIDERMAN') | (spider_pairs2['Target'] == 'SPIDERMAN')]
#print(spider_pairs3.iloc[0:20])

## 1. What does Spider-Man's social network look like?
<br/>
<br/>
Our journey begins by looking at Spider-Man's place in his universe. Using a dataset for Marvel characters with number of co-features in the comic books as connections, the network of their prominent characters is shown here:  
<br/>

![Imgur](https://i.imgur.com/iQy4NC4.gif)
[Data source](http://syntagmatic.github.io/exposedata/marvel/)
Visualized with Gephi.

The coloured groupings above were detected by the asynchronous fluid communities algorithm (a tool in the networkx Python library) and it shows that Spider-Man doesn't really belong with the big teams like the X-Men, the Fantastic Four or the Avengers. However, he does have highly weighted connections to many of their members which reflect his strong historical ties to them. Instead, Spider-Man is the largest node connecting his own community to the rest of the network. The alternate figure above highlights the more important characters for Spider-Man.

According to this data, while Captain America is the most central character with maximum values of degrees, closeness and Eigenvector value, Spider-Man has the highest betweenness measure because he connects so much of his world to the rest of the Marvel Universe.

<br/>
<br/>
So what does his own network look like? The list of characters was pared down to those who have had at least 20 co-features with direct ties to Spider-Man, resulting in 133 nodes. Their community groupings are colored below:
<br/>


![Imgur](https://i.imgur.com/15rdhOn.png)
<p style="text-align: center;"> <i>Ceci n'est pa un diagramme circulaire.<BR>(Tr: This is not a pie chart.)</i>             </p>

<br/>


We start to see that just like many of us, Spider-Man navigates several social groups reflecting the different roles (and literal identities) that he has in his life. In bright green and orange are the tightly connected superhero teams while the few blue nodes are for the "lone wolfs" of the gritty crime-fighting world of the New York streets which includes Daredevil, Luke Cage and the Punisher. In pink are his close friends and family that ground him as Peter Parker. Finally, the circles on the end of the little red spokes represent villains and neutral characters that Spider-Man or Peter encouters on his own.

In [22]:
# for Q2

measures = ['STRENGTH', 'SPEED', 'AGILITY', 'STAMINA', 'DURABILITY', 'INTELLIGENCE']

def gradientbars(bars):
      ax = bars[0].axes
      for bar in bars:
          bar.set_zorder(1)
          bar.set_facecolor("none")
          x,y = bar.get_xy()
          w, h = bar.get_width(), bar.get_height()
          grad = np.atleast_2d(np.linspace(0,1*w/7,256))
          ax.imshow(grad, extent=[x,x+w,y,y+h-0.05], aspect="auto", zorder=1, norm=mpl.colors.NoNorm(vmin=0,vmax=1))
      ax.axis((0.0, 7.2, -0.7, 5.7))

def powerbarfig(name, powers):
    fig, ax = plt.subplots(figsize=(6, 4))     
    bar = ax.barh(measures,powers)
    gradientbars(bar)
    fig.tight_layout()
    plt.title(name, fontsize=22)
    plt.grid(axis='x',color='grey', linestyle='-', linewidth=0.5)
    plt.savefig(os.path.join('images\\'+name+'.png'))
    plt.close()
    
powerbarfig('DAREDEVIL',[2,2,4,4,2,2])    
powerbarfig('HULK', [7,2,2,6,6,5])  
powerbarfig('IRON MAN',[6,6,3,4,5,5])    
powerbarfig('THING', [6,2,2,5,5,2])
powerbarfig('HUMAN TORCH',[2,3,2,4,5,2])    
powerbarfig('THOR', [7,7,2,6,6,2])  
powerbarfig('SPIDER-MAN',[4,3,6,4,3,4])    
powerbarfig('DOCTOR OCTOPUS', [4,3,2,2,2,5])
powerbarfig('GREEN GOBLIN',[4,3,2,2,3,4])    
powerbarfig('KINGPIN', [3,2,2,3,2,4])  
powerbarfig('HOBGOBLIN',[4,3,2,3,3,2])    
powerbarfig('LIZARD', [5,2,5,5,4,2])
powerbarfig('VENOM', [5,2,4,3,3,2])
powerbarfig('CAPTAIN AMERICA', [2,2,5,4,2,3])

<br/>
<br/>

![Imgur](https://i.imgur.com/qjcSsN9.jpg)
A Spider-Man-focused selection from the 1991 series of the Marvel Universe Trading Cards.
<br/>
<br/>


## 2. What makes Spider-Man so special?
<br/>
<br/>
Next, I examined the so-called "Power Ratings" that were assigned to him and other characters and are given on the backs of the 1991 series of the Marvel Universe Trading Cards. One may call these ratings arbitrary, but aren't most ratings exactly that? 

<br/>

![Imgur](https://i.imgur.com/oXFUiIq.png)

Data collected from my very own collection. [Here is a more reliable source.](https://www.tradingcarddb.com/Checklist.cfm/sid/74959/1991-Impel-Marvel-Universe-II?)

<br/>

Other than his strong Agility owing from his radioactive super-spiderpowers, Spider-Man does not seem to be very remarkable. His powers seem less impressive in comparison to the other big superheroes, although he does look more well-rounded. This points to his Everyman quality and likely explains how the character remains the most popular Marvel character for so long, with the public having an equal if not stronger attachment to Peter Parker over the Spider-Man persona. He is a relatable character who starts out as a teenager from Queens trying to balance his priorities between school and family who is then put into extraordinary circumstances and decides to take on the responsibilities that come with it.

On the right hand side, we can note that he usually outmatches his villains. Lucky for him, and New York City.

In [5]:
# Read table of all movies
#mlratings = pd.read_csv('data/ml-latest/ratings.csv') 
# mlratings.shape 
# OUTPUT: (27753444, 4)

# Subset Spider-Man movies
spidermovieIds = [95510, 5349, 8636, 52722, 110553, 122926]
smovie_ratings = mlratings[mlratings['movieId'].isin(spidermovieIds)]
#spidermovie_ratings.shape  
# OUTPUT: (73875, 4)

# Include title and year (from movielens's movies.csv)
smovie_ratings['title'] = pd.Series([])
smovie_ratings['year'] = pd.Series([])
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==95510].index,'title'] = 'The Amazing Spider-Man'
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==95510].index,'year'] = 2012
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==5349].index,'title'] = 'Spider-Man'
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==5349].index,'year'] = 2002
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==8636].index,'title'] = 'Spider-Man 2'
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==8636].index,'year'] = 2004
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==52722].index,'title'] = 'Spider-Man 3'
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==52722].index,'year'] = 2007
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==110553].index,'title'] = 'The Amazing Spider-Man 2'
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==110553].index,'year'] = 2014
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==122926].index,'title'] = 'Spider-Man: Homecoming'
smovie_ratings.loc[smovie_ratings[smovie_ratings.movieId==122926].index,'year'] = 2017

# Write csv, check table
smovie_ratings.to_csv('data\smovies.csv')
#smovie_ratings.head(10)

# this is where I went to do the rest of the data wrangling in Tableau. You can surprisingly accomplish a lot there.

NameError: name 'mlratings' is not defined

<br/>

![Imgur](https://i.imgur.com/H7kSr5z.jpg)
Spider-Man animated series through the decades.
<br/>
<br/>


## 3. Spider-Man swings from one animated series to the next.

<br/>

<br/>
Spider-Man's leap from the comic pages to other forms of media has not been smooth. He starred in a number of successful animated series and still does today. Which one you may fondly remember of depends on your age. 

An ambitious (not to mention expensive) Broadway musical entitled  *Spider-Man: Turn Off the Dark*  closed after a two and a half year run. 


![Imgur](https://i.imgur.com/zzy2Gfb.png)

Source: IMDB

<br/>

![Imgur](https://i.imgur.com/PXPf485.jpg)
Promotional posters for various Spider-Man movies.
<br/>
<br/>




## 3. How has public opinion on the Spider-Man movies changed over time?

<br/>
<br/>
Spider-Man's leap from the comic pages to other forms of media has not been smooth. He starred in a number of successful animated series and still does today. Which one you may fondly remember of depends on your age. 

An ambitious (not to mention expensive) Broadway musical entitled  *Spider-Man: Turn Off the Dark*  closed after a two and a half year run. 


Most famously in recent times, there have been the live action movie adaptations which have garnered great hype that seems to fizzle out, giving us our third interpretation of Peter Parker in less than two decades. MovieLens has a large database of user movie ratings for their service. The ratings for the Spider-Man movies were extracted for the following analysis.

In [31]:
%%html
<div class='tableauPlaceholder' id='viz1550133549098' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;sp&#47;spiderReviews&#47;Ratings4Public&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='spiderReviews&#47;Ratings4Public' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;sp&#47;spiderReviews&#47;Ratings4Public&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550133549098');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1500px';vizElement.style.height='707px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

[Data Source](https://grouplens.org/datasets/movielens/)

<br/>
Evaluating the overall average rating of the movies gives us an idea of their popularity, as you scroll to the right the moving averages through the years paint a fuller story. Each line begins 10 days after the premiere date of the movie. 

*Spider-Man* and *Spider-Man 2* both started with great ratings and hype which stabilized down after a year. *Spider-Man 3* started relatively low and never recovered. Ratings for the first movie of the first reboot started decently without matching the excitement of the earlier movies. While *The Amazing Spider-Man 2* has higher ratings than *Spider-Man 3*, its release seemed to have an amazing effect that the latter did not achieve: it brought down the ratings of all the other movies in the franchise along with it. The ratings for TASM2 have not reached down to SM3 levels yet, although one wonders if it ever will. I marked the premiere date of *Captain America: Civil War* which featured the third actor to take on the Spider-role to see if it sparked any enthusiasm for the character's movies. That effect seems largely absent. Finally, the newest reboot appears to have decent ratings that have not dropped steeply like the first movies, although time will tell how that will go. 

Ratings for the tangential *Into the Spiderverse* movie were not yet available.
*Spider-Man: Far From Home* premieres July 5, 2019.



<br/>
<br/>

# 4. What is the popularity of Spider-Man compared to other superheroes in modern times?

<br/>
<br/>
I used Google Trends to examine the relative popularity of Spider-Man in modern times, with monthly data for search popularity covering the world for the last 15 years. Here, the popularity is specifically in reference to the Fictional Superhero/Character as determined and amalgated by Google rather than as a search term.

In [33]:
%%html
<div class='tableauPlaceholder' id='viz1550137492418' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;q4&#47;q4relPops&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='q4relPops&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;q4&#47;q4relPops&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550137492418');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1000px';vizElement.style.height='727px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

[Data source](https://trends.google.com/trends/)

As expected, there are large spikes whenever a superhero movie is released. 
It is interesting to note that the spikes for the Batman movies have been growing each time while those for Spider-Man have been nearly halved since *Spider-Man 3*. In time periods away from the movie releases, we can observe that Spider-Man and Batman keep decent baselines ahead of the third most popular superhero, Superman. My search for the next most famous Marvel character yielded a few close ones who were far away from the Top 3, with the Hulk shown here.

After averaging their monthly popularities, Spider-Man came ahead of Batman.

<br/>
<br/>


# 5. Spider-Man vs. Batman: can we put different contexts to their relative  popularity on the internet?


<br/>
<br/>


To put Spider-Man and Batman's relative popularities into different contexts, I tried out their names in combination with other search terms. Please note that as a search term, "spiderman" is more frequently used instead of "spider-man" and so I combined the results for both. 

![Imgur](https://i.imgur.com/Xz5UsZ9.png)
<p style="text-align: center;">The relative search histories of "Spider-Man" and its misspelled alternate.</p>


To go even further, the relative popularities of the search terms were colour-coded on a world map, with brighter colours indicating a stronger imbalance. 
Please feel free to use the drop-down bar to examine results.

In [34]:
%%html
<div class='tableauPlaceholder' id='viz1550129622874' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Q5&#47;Q5searchMap&#47;Dashboard2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Q5searchMap&#47;Dashboard2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Q5&#47;Q5searchMap&#47;Dashboard2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550129622874');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1000px';vizElement.style.height='827px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

[Data source](https://trends.google.com/trends/) Please note that for many of the grey-ed out countries, this is reflective of the lack of available data rather than a 50/50 split between Spider-Man and Batman.
<br/>

Some interesting observations include: 
-  while the Batman vs. Spider-Man divide mostly occurs around continent boundaries, more countries are looking for Peter Parker over Bruce Wayne
- Brazil seems to enjoy a very popular Batman meme
- India goes against the rest of the world in their preference for Spider-Man toys and costumes
<br/>
<br/>
<br/>

Altogether, I have shown a few ways to analyze and visualize the popularity of Spider-Man through the ages and across different media, with different insights gained from each one. I hope you have enjoyed exploring the data as much as I did.

<br/>

![Imgur](https://i.imgur.com/GDV0aOo.png)
<p style="text-align: center;">A still from the Spider-Man PlayStation4 game.</p>
<br/>
<br/>

In [29]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle on/off the raw code for this IPython notebook, click <a href="javascript:code_toggle()">here</a>.''')